In [22]:
import trainer.glue_base as glue_base
import models.sparse_token as sparse
import pickle, importlib
importlib.reload(glue_base)
importlib.reload(sparse)
Glue = glue_base.GlueAttentionApproxTrainer

In [23]:
subsets = ["cola","mnli","mrpc","qnli","qqp","rte","sst2","stsb","wnli",]
kss = [0.1, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.999, 'dynamic']
sparse.benchmark_reset()
# subsets = ["mrpc"]
# kss = ['dynamic',]

def get_score(score):
    if 'accuracy' in score:
        return score['accuracy'], "acc"
    first_metric = list(score.keys())[0]
    return score[first_metric], first_metric

results = {}
i = 0
for subset in subsets:
    trainer = Glue(dataset=subset, factor=16, batch_size=-1, device=0)
    trainer.load()
    scores = {}
    metric_name = ""
    #bert_score, metric_name = get_score(trainer.eval_base_model())
    #scores['bert'] = bert_score
    for ks in kss:
        sparse_score, _ = get_score(trainer.eval_sparse_model(ks=ks))
        scores[str(ks)] = sparse_score
        i += 1
        count = len(subsets) * len(kss)
        print(f'{i}/{count}')
    results[f"{subset} ({metric_name})"] = scores

with open('glue_benchmark.pkl', 'wb') as f:
    pickle.dump(results, f)

sparse.benchmark_report()

Trainer: mrpc


Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-7ab73b41110526b8.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-77eb4a03ea1c3167.arrow
Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-c3b7ac3527887b23.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9d

metric score {'accuracy': 0.8394202898550724, 'f1': 0.8823779193205944}
avg occupy 0.5399536083129833
1/1
est_k: (2592, 1833.7129120879106), avg: 0.7074509691697186


In [4]:
import pickle
import pandas as pd

with open('glue_benchmark.pkl', 'rb') as f:
    results = pickle.load(f)

data = []
subsets = list(results.keys())
factors = list(results[subsets[0]].keys())
for factor in factors:
    row = []
    for subset in subsets:
        row.append(results[subset][factor])
        
        

        
    data.append(row)
pd.DataFrame(data, columns=subsets, index=factors)

,cola (matthews_correlation),qnli (acc)
bert,0.533877,0.915431
dynamic,0.028095,0.523522
0.1,0.007996,0.599671
0.25,0.329259,0.841845


In [4]:
trainer = Glue(dataset='qnli', factor=16, batch_size=-1, device=0)
trainer.load()

Trainer: qnli


Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-95cdbfccba10767f.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-a121d057d8b0504c.arrow
Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-81af1b1d0da19e67.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9d

In [5]:
trainer.model.bert = trainer.model_bert
trainer.eval_base_model()

eval: 100%|██████████| 683/683 [00:25<00:00, 26.83it/s]

metric score {'accuracy': 0.9154310818231741}
avg occupy 0.30243502260079336


{'accuracy': 0.9154310818231741}

In [9]:
trainer.seed()
import models.sparse_token as sparse
import transformers.models.bert.modeling_bert as berts
import importlib
importlib.reload(sparse)

wrapped_bert = sparse.ApproxSparseBertModel(trainer.model_bert, approx_bert=trainer.approx_bert, ks=0.1)
sparse_cls_bert = berts.BertForSequenceClassification(trainer.model_bert.config)
sparse_cls_bert.load_state_dict(trainer.model.state_dict())
sparse_cls_bert.bert = wrapped_bert
sparse_cls_bert.to(trainer.device).eval()

trainer.eval_base_model(model = sparse_cls_bert)

eval: 100%|██████████| 683/683 [00:36<00:00, 18.53it/s]


metric score {'accuracy': 0.5996705107084019}
avg occupy 0.30243502260079336


{'accuracy': 0.5996705107084019}